In [ ]:
# accounts
import json
import solana
import time
import sys
import glob
from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey
from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed
from solana.transaction import Transaction, AccountMeta, TransactionInstruction
first_key_in_key_folder = glob.glob("keys/*")[0]
payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])
deployed_program_key = glob.glob("dist/program/*")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])
http_client = Client("https://api.devnet.solana.com")
recent_blockhash = http_client.get_recent_blockhash()
payer_public_key = payer_loaded_account.public_key
program_id = deployed_program_key_account.public_key
GREETING_SEED = "hello"
tx = Transaction()
tx.recent_blockhash = Blockhash(recent_blockhash["result"]["value"]["blockhash"])
greeting_pubkey = PublicKey.create_with_seed(
    from_public_key=payer_public_key, seed=GREETING_SEED, program_id=program_id
)
print(f"Payer Account: {payer_public_key}")
print(f"Greeting Account: {greeting_pubkey}")
print(f"Program ID: {program_id}")

In [146]:
# prepare
import json
import solana
import time
import sys
import glob

from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey

from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed
from solana.transaction import Transaction, AccountMeta, TransactionInstruction

first_key_in_key_folder = glob.glob("keys/*")[0]
payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])

deployed_program_key = glob.glob("dist/program/*")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])

http_client = Client("https://api.devnet.solana.com")
recent_blockhash = http_client.get_recent_blockhash()

payer_public_key = payer_loaded_account.public_key
program_id = deployed_program_key_account.public_key

GREETING_SEED = "hello"

tx = Transaction()
tx.recent_blockhash = Blockhash(recent_blockhash["result"]["value"]["blockhash"])

greeting_pubkey = PublicKey.create_with_seed(
    from_public_key=payer_public_key, seed=GREETING_SEED, program_id=program_id
)

print(f"Payer Account: {payer_public_key}")
print(f"Greeting Account: {greeting_pubkey}")

account_info = http_client.get_account_info(greeting_pubkey)
account_info = account_info.get("result").get("value")

if account_info == None:
    print("Greeting Account Does Not Exist. Creating Now")
    # make dat account
    transaction_instruction = create_account_with_seed(
        params=CreateAccountWithSeedParams(
            from_pubkey=payer_public_key,
            new_account_pubkey=greeting_pubkey,
            base_pubkey=payer_public_key,
            seed={"length": len(GREETING_SEED), "chars": GREETING_SEED},
            lamports=10_000,
            space=4,
            program_id=program_id,
        )
    )
    tx.add(transaction_instruction)
    tx.sign(payer_loaded_account)
    http_client.simulate_transaction(tx)
    http_client.request_airdrop(payer_loaded_account.public_key, 1000000)
    transaction_results = http_client.send_transaction(tx, payer_loaded_account)
    print(transaction_results)
else:
    print(f"Greeting Account Already Created")

Payer Account: 763cSApzBT5scjhSd6BdnkKkMYBhiQmPn2gctswkdkJM
Greeting Account: Bp7wh7XKAU21MTt8FSitiRVrP1jsxCd3X1U95h3j14vZ
Greeting Account Already Created


In [156]:
# Greet
import json
import solana
import time
import sys
import glob

from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey

from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed
from solana.transaction import Transaction, AccountMeta, TransactionInstruction

first_key_in_key_folder = glob.glob("keys/*")[0]
payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])

deployed_program_key = glob.glob("dist/program/*")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])

http_client = Client("https://api.devnet.solana.com")
recent_blockhash = http_client.get_recent_blockhash()

payer_public_key = payer_loaded_account.public_key
program_id = deployed_program_key_account.public_key

GREETING_SEED = "hello"

tx = Transaction()
tx.recent_blockhash = Blockhash(recent_blockhash["result"]["value"]["blockhash"])

greeting_pubkey = PublicKey.create_with_seed(
    from_public_key=payer_public_key, seed=GREETING_SEED, program_id=program_id
)

print(f"Payer Account: {payer_public_key}")
print(f"Greeting Account: {greeting_pubkey}")

tx = Transaction()
tx.recent_blockhash = Blockhash(recent_blockhash["result"]["value"]["blockhash"])


tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=greeting_pubkey, is_signer=False, is_writable=True)],
    program_id=program_id,
    data=b'',
)

tx.add(tx_instruction)

tx.sign(payer_loaded_account)

# http_client.request_airdrop(payer_loaded_account.public_key, 1000000)

http_client.simulate_transaction(tx)

transaction_results = http_client.send_transaction(tx, payer_loaded_account)

print(transaction_results)

Payer Account: 763cSApzBT5scjhSd6BdnkKkMYBhiQmPn2gctswkdkJM
Greeting Account: Bp7wh7XKAU21MTt8FSitiRVrP1jsxCd3X1U95h3j14vZ
{'jsonrpc': '2.0', 'result': '2RGiXDNMTWyQbReEhu8uFkGDjpb287dYmg9qN9xvpqrio4BC9b6t9oJ45TQcqGH9Xdix66oQmYFaZFdH18LP1Gz9', 'id': 4}


In [160]:
# Read

import json
import solana
import time
import sys
import glob
import base64

import solana.system_program as sp

from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey

from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed
from solana.transaction import Transaction, AccountMeta, TransactionInstruction

first_key_in_key_folder = glob.glob("keys/*")[0]
payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])

dzployed_program_key = glob.glob("dist/program/*")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])

http_client = Client("https://api.devnet.solana.com")
recent_blockhash = http_client.get_recent_blockhash()

payer_public_key = payer_loaded_account.public_key
program_id = deployed_program_key_account.public_key

GREETING_SEED = "hello"

tx = Transaction()
tx.recent_blockhash = Blockhash(recent_blockhash["result"]["value"]["blockhash"])

greeting_pubkey = PublicKey.create_with_seed(
    from_public_key=payer_public_key, seed=GREETING_SEED, program_id=program_id
)

print(f"Payer Account: {payer_public_key}")
print(f"Greeting Account: {greeting_pubkey}")

account_info = http_client.get_account_info(greeting_pubkey)
account_data = account_info.get("result").get("value").get("data")

account_greeting_count_data = account_data[0]

number_of_greetings = int.from_bytes(base64.b64decode(account_greeting_count_data), byteorder='little')

print(f"The account has been greeted {number_of_greetings} times!")

Payer Account: 763cSApzBT5scjhSd6BdnkKkMYBhiQmPn2gctswkdkJM
Greeting Account: Bp7wh7XKAU21MTt8FSitiRVrP1jsxCd3X1U95h3j14vZ
The account has been greeted 3 times!
